## Compare FibonacciKAN's ability in function interpolation with MLP

In [21]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
from FibonacciKANLayer import FibonacciKANLayer

In [22]:
# Define target function
def target_function(x):
    y = np.zeros_like(x)
    mask1 = x < 0.5
    y[mask1] = np.sin(20 * np.pi * x[mask1]) + x[mask1] ** 2
    mask2 = (0.5 <= x) & (x < 1.5)
    y[mask2] = 0.5 * x[mask2] * np.exp(-x[mask2]) + np.abs(np.sin(5 * np.pi * x[mask2]))
    mask3 = x >= 1.5
    y[mask3] = np.log(x[mask3] - 1) / np.log(2) - np.cos(2 * np.pi * x[mask3])

    return y

In [23]:
# Define MLP and FibonacciKAN
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.layers(x)


class FibonacciKAN(nn.Module):
    def __init__(self):
        super(FibonacciKAN, self).__init__()
        self.fibonaccikan1 = FibonacciKANLayer(1, 8, 8)
        self.fibonaccikan2 = FibonacciKANLayer(8, 1, 8)

    def forward(self, x):
        x = self.fibonaccikan1(x)
        x = self.fibonaccikan2(x)
        return x

Epoch 1/25000, FibonacciKAN Loss: 1.1709, MLP Loss: 0.6726
Epoch 101/25000, FibonacciKAN Loss: 0.2158, MLP Loss: 0.1915
Epoch 201/25000, FibonacciKAN Loss: 0.2074, MLP Loss: 0.1848
Epoch 301/25000, FibonacciKAN Loss: 0.2037, MLP Loss: 0.1822
Epoch 401/25000, FibonacciKAN Loss: 0.2007, MLP Loss: 0.1799
Epoch 501/25000, FibonacciKAN Loss: 0.1983, MLP Loss: 0.1779
Epoch 601/25000, FibonacciKAN Loss: 0.1955, MLP Loss: 0.1762
Epoch 701/25000, FibonacciKAN Loss: 0.1921, MLP Loss: 0.1743
Epoch 801/25000, FibonacciKAN Loss: 0.1896, MLP Loss: 0.1721
Epoch 901/25000, FibonacciKAN Loss: 0.1876, MLP Loss: 0.1867
Epoch 1001/25000, FibonacciKAN Loss: 0.1867, MLP Loss: 0.1652
Epoch 1101/25000, FibonacciKAN Loss: 0.1869, MLP Loss: 0.1634
Epoch 1201/25000, FibonacciKAN Loss: 0.1861, MLP Loss: 0.1568
Epoch 1301/25000, FibonacciKAN Loss: 0.2014, MLP Loss: 0.1528
Epoch 1401/25000, FibonacciKAN Loss: 0.1860, MLP Loss: 0.1516
Epoch 1501/25000, FibonacciKAN Loss: 0.1859, MLP Loss: 0.1495
Epoch 1601/25000, Fi

In [24]:
# Generate sample data
x_train = torch.linspace(0, 2, steps=500).unsqueeze(1)
y_train = torch.tensor(target_function(x_train))

# Instantiate models
fibonacci_model = FibonacciKAN()
mlp_model = SimpleMLP()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer_fibonacci = torch.optim.Adam(fibonacci_model.parameters(), lr=0.01)
optimizer_mlp = torch.optim.Adam(mlp_model.parameters(), lr=0.007)

# Train the models
epochs = 25000
for epoch in range(epochs):
    optimizer_fibonacci.zero_grad()
    outputs_fibonacci = fibonacci_model(x_train)
    loss_fibonacci = criterion(outputs_fibonacci, y_train)
    loss_fibonacci.backward()
    optimizer_fibonacci.step()

    optimizer_mlp.zero_grad()
    outputs_mlp = mlp_model(x_train)
    loss_mlp = criterion(outputs_mlp, y_train)
    loss_mlp.backward()
    optimizer_mlp.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, FibonacciKAN Loss: {loss_fibonacci.item():.4f}, MLP Loss: {loss_mlp.item():.4f}')

# Test the models
x_test = torch.linspace(0, 2, steps=400).unsqueeze(1)
y_pred_fibonacci = fibonacci_model(x_test).detach()
y_pred_mlp = mlp_model(x_test).detach()

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(x_train.numpy(), y_train.numpy(), 'ro', label='Original Data')
plt.plot(x_test.numpy(), y_pred_fibonacci.numpy(), 'b-', label='FibonacciKAN')
plt.plot(x_test.numpy(), y_pred_mlp.numpy(), 'g--', label='MLP')
plt.title('Comparison of FibonacciKAN and MLP Interpolations f(x)')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()